# Block world simulation

We are given nine blocks on a table as so:

1 \
2 5 \
3 6 8 \
4 7 9 

here 4, 7, 9 are on the table. The table is very wide thus there is enough room to place all blocks on the table. We have to move the blocks such that we observe this pile:

9\
6\
3

where 3 is on the table.

Rules:
- a block can only be moved iff it is not blocked (i.e. you can move 1,5,8 above but not e.g. 7)

We want to achieve this in a minimal number of steps.

Input data:

In [1]:
%%file ./instances/block_simulation_input.lp

block(1..9).
init(
    loc(1,2); loc(2,3); loc(3,4); loc(4,table);
    loc(5,6); loc(6,7); loc(7,table);
    loc(8,9); loc(9,table)
).

final(
    loc(9,6); loc(6,3); loc(3,table)
).

#const horizon=10.

Overwriting ./instances/block_simulation_input.lp


Solver:

In [2]:
%%file ./instances/block_simulation.lp

%initial state at T=0
holds(L,T) :- init(L), T=0.

% all potential moves (legal and illegal)
action(move(B1,B2)) :- block(B1), block(B2), B1!=B2.
action(move(B,table)) :- block(B).
{occurs(A,T): action(A)} = 1 :- T=0..(horizon-1).

% apply actions
holds(loc(B,L),T+1) :- occurs(move(B,L), T).

% only one block can be on top of an other
Top1=Top2 :- holds(loc(Top1, Bottom), T), holds(loc(Top2, Bottom), T), block(Bottom).

% you can only move unblocked blocks
:- occurs(move(B,L),T), holds(loc(_,B),T).

% cannot move to same location
:- occurs(move(B,L),T), holds(loc(B,L),T).

% block can only be at one place
:- #count{L : holds(loc(B,L),T)} != 1, block(B), T=0..horizon.

% inertia: blocks stay where they are when not moved
{holds(loc(B,L),T+1)} :- holds(loc(B,L),T), T=0..horizon-1.

% find minimum number of actions
satisfied(F,T) :- holds(F,T), final(F).
nb_sat(C1,T) :- C1=#count{F: satisfied(F,T)}, T=0..horizon.
nb_fin(C2) :- C2=#count{F: final(F)}.
solved(T) :- nb_sat(C1,T), nb_fin(C2), C1=C2.
:- not solved(_).

#minimize{T: solved(T)}.

% print strategy
strategy(A,T) :- occurs(A,T), T<T_solved, solved(T_solved).
#show strategy/2.

Overwriting ./instances/block_simulation.lp


Below the strategy to achieve the target state is shown. We could also use ASP to solve for every such solution. The finale state is achieved in 7 steps:
1. move 1 on the table
2. move 8 on to 1
3. move 5 on to 8
4. move 2 on the table
5. move 3 on the table
6. move 6 on to 3
7. move 9 on to 6 which is the final state

In [3]:
!clingo ./instances/block_simulation_input.lp ./instances/block_simulation.lp

clingo version 5.6.2
Reading from ./instances/block_simulation_input.lp ...
Solving...
Answer: 1
strategy(move(5,8),0) strategy(move(1,table),1) strategy(move(6,table),2) strategy(move(6,1),3) strategy(move(2,7),4) strategy(move(3,table),5) strategy(move(6,3),6) strategy(move(5,table),7) strategy(move(8,5),8) strategy(move(9,6),9)
Optimization: 10
Answer: 2
strategy(move(5,table),0) strategy(move(8,5),1) strategy(move(1,9),2) strategy(move(2,8),3) strategy(move(3,table),4) strategy(move(6,3),5) strategy(move(1,2),6) strategy(move(9,6),7)
Optimization: 8
Answer: 3
strategy(move(1,table),0) strategy(move(8,1),1) strategy(move(5,8),2) strategy(move(2,table),3) strategy(move(3,table),4) strategy(move(6,3),5) strategy(move(9,6),6)
Optimization: 7
OPTIMUM FOUND

Models       : 3
  Optimum    : yes
Optimization : 7
Calls        : 1
Time         : 0.031s (Solving: 0.01s 1st Model: 0.00s Unsat: 0.00s)
CPU Time     : 0.031s
